In [ ]:
import torch
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import requests
import json
import subprocess

# === Step 1: Load Pretrained ViT (Vision Encoder) ===
vit_model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
vit_model = ViTModel.from_pretrained(vit_model_name)

# === Step 2: Load satu sample gambar ===
image_path = "data/images/CXR1_1_IM-0001-3001.png"  # ganti sesuai dataset kamu
image = Image.open(image_path).convert("RGB")
inputs = feature_extractor(images=image, return_tensors="pt")

# === Step 3: Extract visual embeddings ===
with torch.no_grad():
    outputs = vit_model(**inputs)
    visual_emb = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

print("Visual feature vector size:", len(visual_emb))

# === Step 4: Kirim ke Ollama (LLM) ===
prompt = f"""
You are a medical imaging expert. Based on this encoded visual feature:
{visual_emb[:100]} ...
Generate a brief, professional medical report describing the X-ray.
"""

response = subprocess.run(
    ["ollama", "run", "llama3"],
    input=prompt.encode(),
    stdout=subprocess.PIPE
).stdout.decode()

print("\n🩺 Generated Report:\n", response)


✅ Visual feature vector size: 768

🩺 Generated Report:
 **Medical Imaging Report**

Patient: [Name Redacted]

Date of Examination: [Date Redacted]

Examination Number: [Number Redacted]

Imaging Modality: Digital X-Ray (AP View)

Findings:

The digital X-ray examination reveals a complex fracture pattern involving the left tibia and fibula. The femur appears to be intact, with no significant abnormalities noted.

The fracture morphology is characterized by multiple fragments, including an oblique proximal fragment, a distal metaphyseal fragment, and several smaller cortical fragments. There is evidence of comminution at the site of the fracture, suggesting significant trauma.

No other notable radiographic findings were identified, including no signs of osteomyelitis or septic arthritis. The surrounding soft tissues appear normal.

**Interpretation:**

The digital X-ray examination suggests a high-energy trauma mechanism, consistent with a significant fall or motor vehicle accident. Fu

In [3]:
import pandas as pd

df = pd.read_csv("data/iu_xray_dataset.csv")
results = []

for idx, row in df.head(5).iterrows():  # test dulu 5 data
    image_path = row['image']
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = vit_model(**inputs)
        visual_emb = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

    prompt = f"""
    You are a medical imaging expert. Based on this encoded visual feature:
    {visual_emb[:100]} ...
    Generate a concise, professional medical report describing the X-ray.
    """

    response = subprocess.run(
        ["ollama", "run", "llama3"],
        input=prompt.encode(),
        stdout=subprocess.PIPE
    ).stdout.decode()

    results.append({"image": image_path, "generated_report": response})

pd.DataFrame(results).to_csv("data/generated_reports.csv", index=False)
